In [ ]:
import redcap
import pandas as pd
from temba_client.v2 import TembaClient
from temba_client.exceptions import TembaRateExceededError, TembaTokenError, TembaHttpError, TembaNoSuchObjectError
import math
import numpy as np
import json
from support_functions import * 
#from temba_client.values.constants import Value


In [ ]:
def upload_contacts(redcap,
                    rapidpro,
                    datafield, 
                    msisdn_field_name,
                    data_fields):
    import phonenumbers
    import types
    for index,row in data_records_flat.iterrows():
        wa_number = row[msisdn_field_name]
        contact_name = row['name'] + ' ' + row['surname']
        print(wa_number)
        print(contact_name)
        try:
            f_wa_number = phonenumbers.parse(wa_number, None)
            phone_number = ('tel:'+phonenumbers.format_number(f_wa_number,
                                                              phonenumbers.PhoneNumberFormat.E164))
            print(phone_number)
            contacts = rapidpro.client.get_contacts(urn = phone_number)
            #if the contact exists then make sure it is in the correct group
            if contacts.first() is None:
                print('contact doesnt exist- creating')
                rapidpro.client.create_contact(urns = [phone_number], name = contact_name )
                rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
                rapidpro.add_fields_to_contact(phone_number, data_fields)
            else:
                print('contact already exists- just updating')
                rapidpro.client.update_contact(phone_number, name = contact_name)
                rapidpro.add_fields_to_contact(phone_number, data_fields)
        except:
            print('failed')
            print(wa_number)

Define classes to contain the different methods that will be used

In [ ]:
redcap = RedCap()
rapidpro = rapidPro()

Here we can give the names of specific forms and get the missing fields from those

In [ ]:
missing = []
form_names = ['asos2_investigator_phase1','asos2_investigator_phase2']
ignore_fields = ['']
for form in form_names:
    missing_temp = []
    redcap.read_redcap_credentials_file(credential_file = 'redcapcredentials_ASOS2_Investigator_Registration.json',
                                        form = form,
                                       ignore_fields = ignore_fields)
    data_records = redcap.get_records()
    data_records_flat = redcap.get_records_flat()
    data_metadata = redcap.get_metadata()
    expected = redcap.get_expected_fields()
    completed = redcap.get_completed_fields()
    missing_temp = redcap.get_missing_fields()
    missing.append(missing_temp)


Get the record data and metadata

In [ ]:
missing_fields = pd.DataFrame.from_dict(missing).T
missing_fields.columns = form_names

In [ ]:
credential_files = ['redcapcredentials_ASOS2_Investigator_Registration_French.json','redcapcredentials_ASOS2_Investigator_Registration.json']

for redcap_project_credentials in credential_files:
    redcap.read_redcap_credentials_file(credential_file = redcap_project_credentials,
                                    form = form,
                                   ignore_fields = [''])
    rapidpro.read_rapidpro_credentials_file(credential_file = 'rapidprocredentials.json')
    data_records = redcap.get_records()
    data_records_flat = redcap.get_records_flat()
    data_metadata = redcap.get_metadata()
    data_fields = {'project_name' : redcap_project_credentials.replace('.json','')}
    upload_contacts(redcap,rapidpro,data_records_flat,'whatsapp_number',data_fields)

Next we need to parse the phone number from the Redcap database and check it works -- it needs to be converted into the correct format for the message sender

Here we update all the contacts as necessary